# JupyterDock
To run the following cells, refered to JupyterDock gitub repository [https://github.com/AngelRuizMoreno/Jupyter_Dock](https://github.com/AngelRuizMoreno/Jupyter_Dock).
Create virtual environment and install required libraries and activate the environement

# **Load Libraries**

In [ ]:
from pymol import cmd
import py3Dmol
import pandas as pd

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem

from joblib import Parallel, delayed

from tqdm import tqdm

import glob
import sys, os, random
sys.path.insert(1, '/home/juni/Downloads/Jupyter_Dock-main/Jupyter_Dock-main/utilities/')

from utils import getbox, generate_ledock_file, dok_to_sdf

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

# **Docking box definition**

In [ ]:
cmd.load(filename='Path to receptor pdb file',format='pdb',object='prot') #Not needed but as reference of the system
cmd.load(filename='Path to ligand pdb file',format='pdb',object='lig')

center,size=getbox(selection='lig',extending=6.0,software='vina')
cmd.delete('all')
print(center)
print(size)

In [ ]:
os.chdir('Path to mol2 files')

In [ ]:
input_files = glob.glob('*.mol2')

# **Define function for docking to utlize all CPUs**

In [ ]:
def run_smina(input_file):
    receptor_path = ['Path to receptor pdb file']
    input_path = ['Path to ligand files' + str(input_file)]
    output_path = ['Path for output docked files' + str(input_file)[:-5] +  '_docked.sdf']
    smina_command = ['Path to smina code -r '+ str(receptor_path[0])+ 
                    ' -l '+str(input_file)+ ' -o '+str(output_path[0])+ ' --center_x ' +str(center['center_x'])+
                    ' --center_y '+str(center['center_y'])+ ' --center_z '+str(center['center_z'])+ 
                    ' --size_x '+ str(size['size_x'])+ ' --size_y '+ str(size['size_y'])+ ' --size_z '+ str(size['size_z'])+ ' --exhaustiveness 8 --num_modes 1']
    os.system(smina_command[0])

In [ ]:
%%time
Parallel(n_jobs = 40, backend = 'multiprocessing')(delayed(run_smina)(input_file) for input_file in tqdm(input_files))